In [16]:
import os
import pandas as pd
import matplotlib.pyplot as plt

In [32]:
path_drep = "../drep_genomes/OUTPUT/rep_genomes/"
drep_samples = {}
for directory in os.listdir(path_drep):
    path_2 = path_drep + directory
    if os.path.isdir(path_2):
        for directory_2 in os.listdir(path_2):
            path_3 = path_2 + "/" + directory_2
            if os.path.isdir(path_3):
                for directory_3 in os.listdir(path_3):
                    path_4 = path_3 + "/" + directory_3 + "/"
                    if os.path.isdir(path_4):
                        for directory_samp in os.listdir(path_4):
                            samp_dir = path_4 + directory_samp
                            if os.path.isdir(samp_dir):
                                drep_samples[directory_samp] = path_4

In [4]:
prodigal_paths = []
for sample in drep_samples.keys():
    sample_path = drep_samples[sample] + sample
    prodigal_path = sample_path + "/" + sample + ".prodigal.faa.gz"
    prodigal_paths.append(prodigal_path)

In [22]:
num_samples = len(prodigal_paths)
final_cycle_ind = num_samples //1000
path_input = "../clusters/INPUT/mmseqs2_testdb_input.faa.gz"

In [34]:
cat_cmd_1 = "cat "
for i in range(1000):
    cat_cmd_1 += prodigal_paths[i] + " "
cat_cmd_1 += " > " + path_input
os.system(cat_cmd_1)

for i in range(1, final_cycle_ind):
    cat_cmd = "cat "
    for j in range(i * 1000, (i + 1) * 1000):
        
        cat_cmd += prodigal_paths[j] + " "
    cat_cmd += " >> " + path_input
    os.system(cat_cmd)

cat_cmd = "cat "
for i in range(final_cycle_ind * 1000, num_samples):
    cat_cmd += prodigal_paths[i] + " "
cat_cmd += " >> " + path_input
os.system(cat_cmd)
        

In [ ]:
os.system(cat_cmd)

In [13]:
backup_cmd = "gsutil cp " + path_input + " gs://jluo_bucket/ggdb/clusters"
os.system(backup_cmd)

'gsutil cp ../clusters/INPUT/mmseqs2_testdb_input.faa.gz gs://jluo_bucket/ggdb/clusters'

In [35]:
path_temp = "ggdb_multisql_errorlog.csv"

In [ ]:
ls

8156401/     genegraph.db                ggdb_summary_stats.csv
LICENSE.txt  genegraphdb/                scrap_notebook.ipynb
MANIFEST.in  genegraphdb.egg-info/       scrap_notebook.py
README.md    ggdb_load_stats.csv         setup.py
build/       ggdb_multisql_errorlog.csv


In [47]:
in_df = pd.read_csv(path_temp, header=None)
in_df = in_df.iloc[3928:].set_index(0)
#in_df.to_csv(path_temp, header = False)

## choose representatives for stringent clusters

In [10]:
# iterate through unique pid (cluster representatives) in tsv
# explore data (distribution of cluster sizes, do any representatives have >1 *)
# map each pid to sampleid
# get coordinates + length
# get sequence > # of *
# priority is (1) no * (2) length 

path_tsv = "../clusters/OUTPUT/stringent/tmp/clu_cluster_10k.tsv"
cluster_df = pd.read_csv(path_tsv, sep="\t", header = None)

In [17]:
rep_counts = cluster_df.groupby(0).count()[1]

In [31]:
all_pids_s = cluster_df[1]


In [33]:
drep_samples

{'EARTH_3300014595_4': '../drep_genomes/OUTPUT/rep_genomes/A011/B022/C007/',
 'EARTH_3300014595_1': '../drep_genomes/OUTPUT/rep_genomes/A011/B022/C007/',
 'EARTH_3300014613_7': '../drep_genomes/OUTPUT/rep_genomes/A011/B022/C008/',
 'EARTH_3300014604_3': '../drep_genomes/OUTPUT/rep_genomes/A011/B022/C008/',
 'EARTH_3300014604_4': '../drep_genomes/OUTPUT/rep_genomes/A011/B022/C008/',
 'EARTH_3300014613_19': '../drep_genomes/OUTPUT/rep_genomes/A011/B022/C008/',
 'EARTH_3300014613_10': '../drep_genomes/OUTPUT/rep_genomes/A011/B022/C008/',
 'EARTH_3300014603_4': '../drep_genomes/OUTPUT/rep_genomes/A011/B022/C008/',
 'EARTH_3300014602_1': '../drep_genomes/OUTPUT/rep_genomes/A011/B022/C008/',
 'EARTH_3300014961_5': '../drep_genomes/OUTPUT/rep_genomes/A011/B022/C030/',
 'EARTH_3300014963_29': '../drep_genomes/OUTPUT/rep_genomes/A011/B022/C030/',
 'EARTH_3300014956_14': '../drep_genomes/OUTPUT/rep_genomes/A011/B022/C030/',
 'EARTH_3300014962_1': '../drep_genomes/OUTPUT/rep_genomes/A011/B022/C03